# Breakthru
Player 1: 
- Frota: 8 navios amarelos e 1 navio-almirante localizados no centro do tabuleiro.
- Objetivo: Mover o navio-almirante para o perímetro do tabuleiro.

Player 2:
- Frota: 12 navios cinzas localizados nas bordas do tabuleiro.
- Objetivo: Capturar o navio-almirante. 

Gameplay:
- É escolhido aleatóriamente quem será o amarelo e quem será o cinza.
- O amarelo escolhe quem começa.
- Possibilidade de jogada 1: O jogador pode se mover pra qualquer espaço vazio verticalmente ou horizontalmente (Tal qual a torre do xadrez).
- Possibilidade de jogada 2: Mover um quadrado na diagonal para capturar uma peça do oponente (Tal qual um peao do xadrez).

# Tabuleiro
<div>
<img src="./tabuleiro.png" width="400"/>
</div>



In [29]:
import tkinter as tk

class BoardGameGUI:
    def __init__(self, master, lado, player, computer, begin):
        self.master = master
        self.player = player
        self.computer = computer
        self.lado = lado
        self.turn = begin
        self.minimax_calls = 0
        self.create_board()

    def create_board(self):
        self.buttons = []
        self.board = [[None for _ in range(self.lado)] for _ in range(self.lado)]  # Representação do estado do tabuleiro
        for row in range(self.lado):
            button_row = []
            for col in range(self.lado):
                button = tk.Button(self.master, width=4, height=2, command=lambda r=row, c=col: self.on_button_click(r, c), text="( " + str(row) + ", " + str(col) + " )")
                button.grid(row=row, column=col)
                button_row.append(button)
            self.buttons.append(button_row)
        
        for i in range(2,5):
            self.place_piece(0, i, "Cinza")
            self.place_piece(i, 0, "Cinza")
            self.place_piece(i, 6, "Cinza")
            self.place_piece(6, i, "Cinza")

            self.place_piece(2, i, "Amarelo")
            self.place_piece(i, 2, "Amarelo")
            self.place_piece(i, 4, "Amarelo")
            self.place_piece(4, i, "Amarelo")

        self.place_piece(3, 3, "N")

    def on_button_click(self, row, col):
        if self.board[row][col] is not None:
            self.selected_piece = (row, col)
        elif self.selected_piece is not None:
            selected_row, selected_col = self.selected_piece
            self.move_piece(selected_row, selected_col, row, col)
            self.selected_piece = None     
        
    def place_piece(self, row, col, piece):
        self.board[row][col] = piece
        button = self.buttons[row][col]
        if piece == "Amarelo":
            button.config(bg="gold1")
        elif piece == "N":
            button.config(bg="goldenrod")
        elif piece == "Cinza":
            button.config(bg="gray60")
        else:
            button.config(bg="SystemButtonFace")
    
    def move_piece(self, row, col, new_row, new_col):
        piece = self.board[row][col]

        self.place_piece(row,col,None)              # Remove a peça da posição antiga
        self.place_piece(new_row,new_col,piece)     # Coloca a peça na nova posição

        if self.turn == "Amarelo":
            self.turn = "Cinza"
        else:
            self.turn = "Amarelo"

    def get_possible_moves(self, board, player):
        possible_moves = []

        for row in range(self.lado):
            for col in range(self.lado):
                if board[row][col] == player:
                    # Gerar movimentos para cima até o fim do tabuleiro
                    for i in range(row-1, -1, -1):
                        if board[i][col] == None:
                            possible_moves.append(((row,col),(i, col)))
                        else:
                            break

                    # Gerar movimentos para baixo até o fim do tabuleiro
                    for i in range(row+1, self.lado):
                        if board[i][col] == None:
                            possible_moves.append(((row,col),(i, col)))
                        else:
                            break

                    # Gerar movimentos para a esquerda até o fim do tabuleiro
                    for i in range(col-1, -1, -1):
                        if board[row][i] == None:
                            possible_moves.append(((row,col),(row, i)))
                        else:
                            break

                    # Gerar movimentos para a direita até o fim do tabuleiro
                    for i in range(col+1, self.lado):
                        if board[row][i] == None:
                            possible_moves.append(((row,col),(row, i)))
                        else:
                            break
                        
        return possible_moves

    def make_best_move_minimax(self):
        best_score = -float("inf")
        best_move = None

        for move in self.get_possible_moves(self.board, self.computer):
            new_board = self.make_move(self.board, move, self.computer)
            score = self.minimax(new_board, False,3)
            if score > best_score:
                best_score = score
                best_move = move

        if best_move is not None:
            (start_row, start_col), (end_row, end_col) = best_move
            self.move_piece(start_row, start_col, end_row, end_col)
            
        return best_move
    
    def minimax(self, board, is_maximizing, depth):
        self.minimax_calls += 1

        if self.check_winner(board) is not None or depth == 0:
            return self.get_score(board)

        if is_maximizing:
            best_score = -float("inf")
            for move in self.get_possible_moves(board, self.computer):
                new_board = self.make_move(board, move, self.computer)
                score = self.minimax(new_board, False, depth-1)
                best_score = max(best_score, score)
            return best_score

        else:
            best_score = float("inf")
            for move in self.get_possible_moves(board, self.player):
                new_board = self.make_move(board, move, self.player)
                score = self.minimax(new_board, True, depth-1)
                best_score = min(best_score, score)
            return best_score
        
    def make_move(self, board, move, player):
        new_board = [row[:] for row in board]  # Copy the board
        (start_row, start_col), (end_row, end_col) = move
        new_board[end_row][end_col] = new_board[start_row][start_col]
        new_board[start_row][start_col] = None
        return new_board
        
    def check_winner(self, board):
        # Checar se o Amarelo chegou no perimetro (Navio fugiu)
        for lado in range(self.lado):
            if board[0][lado] == "N":
                return "Amarelo"
            if board[6][lado] == "N":
                return "Amarelo"
            if board[lado][0] == "N":
                return "Amarelo"
            if board[lado][6] == "N":
                return "Amarelo"
            
        # Checar se nao tem N no tabuleiro (Navio capturado)
        for row in range(self.lado):
            for col in range(self.lado):
                if board[row][col] == "N":
                    return None
                
        return "Cinza"
    
    def get_score(self, board):
        winner = self.check_winner(board)
        if winner == self.computer:
            return 1
        if winner == self.player:
            return -1
        return 0
    
    def disable_buttons(self):
        for row in self.buttons:
            for button in row:
                button.config(state="disabled")
    
    def enable_buttons(self):
        # Habilita os botões que nao sao do oponente
        for row in range(self.lado):
            for col in range(self.lado):
                if self.board[row][col] != self.computer:
                    self.buttons[row][col].config(state="normal")
        
        

In [30]:
import random
import time

def main():
    window = tk.Tk()
    window.title("Breakthru")
    lado = 7
    text = ""

    if random.choice([1, 2]) == 1:
        player = "Cinza"
        computer = "Amarelo"
        text = "Computador é o Amarelo \nVocê é o Cinza"
    else:
        player = "Amarelo"
        computer = "Cinza"
        text = "Computador é o Cinza \nVocê é o Amarelo"

    info_label = tk.Label(window, text=text)
    info_label.grid(row=lado, columnspan=lado)
    
    begin = random.choice([player, computer])
    game = BoardGameGUI(window, lado, player, computer, begin)
    game.disable_buttons()
    window.update()
    time.sleep(2)

    while True:
        window.update()
        
        if game.turn == player:
            info_label.config(text="Sua vez")
            game.enable_buttons()
        else:
            info_label.config(text="Vez do computador")
            game.disable_buttons()
            window.update()
            game.make_best_move_minimax()
            window.update()

        winner = game.check_winner(game.board)
        if winner is not None:
            game.disable_buttons()
            if winner == player:
                info_label.config(text="Você ganhou!")
            else:
                info_label.config(text="Computador ganhou!")
            window.update()
            time.sleep(2)
            break
        
    window.destroy()

    

if __name__ == "__main__":
    main()